In [1]:
import numpy as np
import nibabel as nib
import glob

import matplotlib.pyplot as plt
from tiffile import imsave
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
import os


In [2]:
t2_list = sorted(glob.glob('D:/brats/BraTS2021_train/*/*t2.nii.gz'))
t1_list = sorted(glob.glob('D:/brats/BraTS2021_train/*/*t1.nii.gz'))
t1ce_list = sorted(glob.glob('D:/brats/BraTS2021_train/*/*t1ce.nii.gz'))
flair_list = sorted(glob.glob('D:/brats/BraTS2021_train/*/*flair.nii.gz'))
mask_list = sorted(glob.glob('D:/brats/BraTS2021_train/*/*seg.nii.gz'))

In [3]:
len(t2_list)

1251

In [ ]:
#no
for img in range(len(t2_list)):   
    print("Now preparing image and masks number: ", img)

    image= nib.load(flair_list[img])

    #Load images in nifti format and stacking them to make a multi-modal file

    temp_image_flair=nib.load(flair_list[img]).get_fdata()
    temp_image_flair=scaler.fit_transform(temp_image_flair.reshape(-1, temp_image_flair.shape[-1])).reshape(temp_image_flair.shape)

    temp_image_t1=nib.load(t1_list[img]).get_fdata()
    temp_image_t1=scaler.fit_transform(temp_image_t1.reshape(-1, temp_image_t1.shape[-1])).reshape(temp_image_t1.shape)

    temp_image_t1ce=nib.load(t1ce_list[img]).get_fdata()
    temp_image_t1ce=scaler.fit_transform(temp_image_t1ce.reshape(-1, temp_image_t1ce.shape[-1])).reshape(temp_image_t1ce.shape)

    temp_image_t2=nib.load(t2_list[img]).get_fdata()
    temp_image_t2=scaler.fit_transform(temp_image_t2.reshape(-1, temp_image_t2.shape[-1])).reshape(temp_image_t2.shape)
    
    temp_combined_images = np.stack([temp_image_flair, temp_image_t1, temp_image_t1ce, temp_image_t2], axis=3)

    #load mask and adjust the labels as per the monai format
   
    temp_mask=nib.load(mask_list[img]).get_fdata()
    temp_mask=temp_mask.astype(np.uint8)
    temp_mask[temp_mask==4] = 3  
    temp_mask[temp_mask==1] = 4
    temp_mask[temp_mask==2] = 5
    temp_mask[temp_mask==4] = 2
    temp_mask[temp_mask==5] = 1

    final_mask = nib.Nifti1Image(temp_mask, image.affine)
    final_image = nib.Nifti1Image(temp_combined_images, image.affine)

    #save the stacked images and corresponding multi-label mask

    nib.save(final_image, 'D:/brats/sample/images/BRATS_'+str(img)+'.nii.gz')
   
    nib.save(final_mask, 'D:/brats/sample/masks/BRATS_'+str(img)+'.nii.gz')
